In [2]:
import numpy as np
from scipy.stats import norm, multivariate_normal
import pandas as pd
from scipy.optimize import newton

# Problems from McDonald (2013)

## 16.7

The formula for Delta-Gamma-Theta approximation:

$$
\begin{aligned}
C(S_t, t) \approx C(S, 0) &+ \Delta(S, 0) \times [S_t - S] \\
&+ \frac{1}{2} \times \Gamma(S, 0) \times [S_t - S]^2 \\
&+ \theta(S, 0) \times t
\end{aligned}
$$

First, we can compute $d_1$ and $d_2$:

$$d_1 = \frac{\ln(\frac{S}{K}) + (r - \delta + \frac{\sigma^2}{2}) \times T}{\sigma \times \sqrt{T}} = \frac{\ln(\frac{40}{40}) + (0.08 - 0 + \frac{0.3^2}{2}) \times 0.49315}{0.3 \times \sqrt{0.49315}} = 0.2926$$

$$d_2 = d_1 - \sigma \times \sqrt{T} = 0.2926 - 0.3 \times \sqrt{0.49315} = 0.08193$$

Now using the Black-Scholes formula we compute the Greeks at Day 0.

$$\Delta = e^{-\delta \times T} \times N(d_1) = e^{-0 \times 0.49315} \times N(0.2926) = 0.6151$$

$$\Gamma = \frac{N'(d_1)}{S \times \sigma \times \sqrt{T}} = \frac{\frac{1}{\sqrt{2\pi}}e^{-0.2926^2/2}}{40 \times 0.3 \times \sqrt{0.49315}}=0.4536$$

$$\theta = \frac{-S \times N'(d_1)\times \sigma}{2\sqrt{T}} - r \times K \times e^{-rT} \times N(d_2)$$
$$= \frac{-40 \times N'(0.2926)\times 0.3}{2\sqrt{0.49315}} - 0.08 \times 40 \times e^{-0.08 \times 0.49315} \times N(0.08193) = -0.0134 \text{ per day}$$

Initial Call Premium:
$$C(S,0) = S \times e^{-\delta \times T} \times N(d_1) - K \times e^{-r \times T} \times N(d_2) = \$4.122$$

In [29]:
def black_scholes_call(S, K, T, r, sigma, delta_div):
    d1 = (np.log(S / K) + (r - delta_div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * np.exp(-delta_div * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def call_delta(S, K, T, r, sigma, delta):
    d1 = (np.log(S / K) + (r - delta + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return np.exp(-delta * T) * norm.cdf(d1)

def call_gamma(S, K, T, r, sigma, delta):
    d1 = (np.log(S / K) + (r - delta + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

def call_theta(S, K, T, r, sigma, delta):
    d1 = (np.log(S / K) + (r - delta + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    theta = - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    return theta/365

S = 40
K = 40
sigma = 0.30
r = 0.08
delta_div = 0
T = 180/365

delta = call_delta(S, K, T, r, sigma, delta_div)
gamma = call_gamma(S, K, T, r, sigma, delta_div)
theta = call_theta(S, K, T, r, sigma, delta_div)
initial_call_premium = black_scholes_call(S,K,T,r,sigma, delta_div)
# print(f'initial_call_premium={initial_call_premium}\n')
# print(f'delta={delta}\n')
# print(f'gamma={gamma}\n')
# print(f'theta={theta}\n')

days = [1, 5, 25]
stock_prices = np.arange(36, 44.25, 0.25)
results = {}

for day in days:
    results[day] = {}
    for price in stock_prices:
        predicted_premium = initial_call_premium + delta * (price - S) + 0.5 * gamma * (price - S)**2 + theta * day
        actual_premium = black_scholes_call(price, K, T - day/365, r, sigma, delta_div)
        results[day][price] = {'Predicted': predicted_premium, 'Actual': actual_premium}

# Build a nested dictionary with stock prices as keys
table = {}
for day, price_dict in results.items():
    for price, values in price_dict.items():
        if price not in table:
            table[price] = {}
        table[price][(day, 'Predicted')] = values['Predicted']
        table[price][(day, 'Actual')] = values['Actual']

df = pd.DataFrame.from_dict(table, orient='index')
df.index.name = 'Stock Price'
# Sort columns by day (as integer) and then by subcolumn order (Predicted first)
df = df.sort_index(axis=1, level=0)
df.index = df.index.map(lambda x: f"{x:.2f}")

# Print as LaTeX
print(df.to_latex(multicolumn=True, float_format="%.4f"))

\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{2}{r}{1} & \multicolumn{2}{r}{5} & \multicolumn{2}{r}{25} \\
 & Actual & Predicted & Actual & Predicted & Actual & Predicted \\
Stock Price &  &  &  &  &  &  \\
\midrule
36.00 & 2.0365 & 2.0108 & 1.9921 & 1.9571 & 1.7660 & 1.6884 \\
36.25 & 2.1424 & 2.1207 & 2.0971 & 2.0669 & 1.8663 & 1.7982 \\
36.50 & 2.2515 & 2.2333 & 2.2053 & 2.1796 & 1.9701 & 1.9108 \\
36.75 & 2.3637 & 2.3488 & 2.3168 & 2.2951 & 2.0773 & 2.0264 \\
37.00 & 2.4792 & 2.4672 & 2.4315 & 2.4134 & 2.1880 & 2.1447 \\
37.25 & 2.5979 & 2.5883 & 2.5495 & 2.5346 & 2.3020 & 2.2659 \\
37.50 & 2.7198 & 2.7123 & 2.6707 & 2.6586 & 2.4195 & 2.3899 \\
37.75 & 2.8449 & 2.8392 & 2.7951 & 2.7854 & 2.5403 & 2.5167 \\
38.00 & 2.9730 & 2.9689 & 2.9226 & 2.9151 & 2.6645 & 2.6464 \\
38.25 & 3.1044 & 3.1014 & 3.0534 & 3.0476 & 2.7921 & 2.7789 \\
38.50 & 3.2387 & 3.2367 & 3.1872 & 3.1830 & 2.9231 & 2.9142 \\
38.75 & 3.3762 & 3.3749 & 3.3241 & 3.3211 & 3.0573 & 3.0524 \\
39.00 & 3.5167 & 3.5159 

The approximation is most accurate when the change in the underlying stock price and time are both small. As these increase, the approximation becomes less accurate.

## 17.11

### a.

We are given: $S_0 = \$40$, $\sigma = 0.30$, $\delta = 0$, $K = \$40$, $r = 0.08$, and $T = 2$ years. We'll use the Black-Scholes formula to determine the price of the call.

$d_1 = \frac{\ln(\frac{S_0}{K}) + (r - \delta + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}} = \frac{\ln(\frac{40}{40}) + (0.08 - 0 + \frac{0.3^2}{2})2}{0.3\sqrt{2}} = 0.49497$

$d_2 = d_1 - \sigma\sqrt{T} = 0.49497 - 0.3\sqrt{2} = 0.0707$

Now, the Call option price is

$C = S_0 e^{-\delta T}N(d_1) - Ke^{-rT}N(d_2) = 40 \times e^{-0 \times 2} \times N(0.49497) - 40 \times e^{-0.08 \times 2} \times N(0.0707) = 40 \times 0.6897 - 40 \times e^{-0.16} \times 0.5282 = \$9.864$

### b.

This compound call gives the right to buy the 2-year call option (from part a) in 1 year for $2. We will exercise the call option at time $t=1$ when the value of the option at that time is larger than $\$2$.

We must obtain the price $S_1$ such that, when entered to the black-scholes formula, will give us a call option price of $2.

In [30]:
def black_scholes_call(S, K, T, r, sigma, delta_div):
    d1 = (np.log(S / K) + (r - delta_div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * np.exp(-delta_div * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def find_S1(target_price, K, T, r, sigma, delta_div):

    def price_difference(S):
        return black_scholes_call(S, K, T, r, sigma, delta_div) - target_price

    S1 = newton(price_difference, 40)
    return S1

K = 40
T = 1
r = 0.08
sigma = 0.30
delta_div = 0
target_price = 2

S1_star = find_S1(target_price, K, T, r, sigma, delta_div)
print(f"After calculating in Python: The stock price S1 at which the compound call is exercised: ${S1_star:.2f}")


After calculating in Python: The stock price S1 at which the compound call is exercised: $31.72


So, we will want to execute the compound option when $S_1 > \$31.72$.

### c.

We are pricing a call on a call. We have an option that gives the right after $t=1$ to buy a call for $x = \$2$ and with expiration date $T = 2$ years. We already computed $S_1^*=\$35.89$, at which price, we are going to start paying for the call. The price for the compound call is found by using Geske's Formula.

$C_c = S_0e^{-\delta T}M(a_1, b_1; \rho) - Ke^{-rT}M(a_2, b_2; \rho) - e^{-r t}xN(a_2)$

Where:
$a_1 = \frac{\ln(\frac{S_0}{S^*_1}) + (r-\delta+\frac{\sigma^2}{2})t}{\sigma\sqrt{t}}=\frac{\ln(\frac{40}{35.89}) + (0.08 - 0 + \frac{0.3^2}{2})1}{0.3\sqrt{1}} = 0.76844$
$a_2 = a_1 - \sigma\sqrt{t} = 0.76844 - 0.3 = 0.46844$

$b_1 = \frac{\ln(\frac{S_0}{K}) + (r - \delta + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}=\frac{\ln(\frac{40}{40}) + (0.08 - 0 + \frac{0.3^2}{2})2}{0.3\sqrt{2}}=0.49497$
$b_2 = b_1 - \sigma\sqrt{T} = 0.0707$

$\rho = \sqrt{\frac{t}{T}} = \sqrt{\frac{1}{2}} = 0.7071$

$M(a,b, \rho)$ is the cumulative bivariate normal distribution.




In [31]:
def compound_call(S0, K, T, t, r, sigma, delta, x, S_star):
    a1 = (np.log(S0 / S_star) + (r - delta + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    a2 = a1 - sigma * np.sqrt(t)
    b1 = (np.log(S0 / K) + (r - delta + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    b2 = b1 - sigma * np.sqrt(T)
    rho = np.sqrt(t / T)

    mvn1 = multivariate_normal(mean=[0, 0], cov=[[1, rho], [rho, 1]])
    M1 = mvn1.cdf([a1, b1])

    mvn2 = multivariate_normal(mean=[0, 0], cov=[[1, rho], [rho, 1]])
    M2 = mvn2.cdf([a2, b2])


    compound_c = S0 * np.exp(-delta * T) * M1 - K * np.exp(-r * T) * M2 - np.exp(-r * t) * x * norm.cdf(a2)
    return compound_c

S0 = 40
K = 40
T = 2
t = 1
r = 0.08
sigma = 0.30
delta = 0
x = 2
S_star = 35.89
Cc = compound_call(S0, K, T, t, r, sigma, delta, x, S_star)
print(f"After calculating in Python: Price of compound call option is: ${Cc:.4f}")

After calculating in Python: Price of compound call option is: $7.8419


### d.

Here, we have a put on a call. We have an option that gives the right after $t = 1$ to sell a call option for $x = \$2$. We want to find the price of the compound put. The option with expiration at $T=2$. Geske's formula for a compound put is:

$P_c = Ke^{-rT}M(-a_2, b_2; -\rho) - S_0e^{-\delta T}M(-a_1, b_1; -\rho) + e^{-rt}xN(-a_2)$

Where $a_1, a_2, b_1, b_2, \rho$ are the same as on part c.

In [33]:
def compound_put(S0, K, T, t, r, sigma, delta, x, S_star):
    a1 = (np.log(S0 / S_star) + (r - delta + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    a2 = a1 - sigma * np.sqrt(t)
    b1 = (np.log(S0 / K) + (r - delta + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    b2 = b1 - sigma * np.sqrt(T)
    rho = np.sqrt(t / T)

    # note the -rho for the compound put
    mvn1 = multivariate_normal(mean=[0, 0], cov=[[1, -rho], [-rho, 1]])
    M1 = mvn1.cdf([-a1, b1])

    mvn2 = multivariate_normal(mean=[0, 0], cov=[[1, -rho], [-rho, 1]])
    M2 = mvn2.cdf([-a2, b2])

    compound_p =  K * np.exp(-r * T) * M2 - S0 * np.exp(-delta * T) * M1 + np.exp(-r * t) * x * norm.cdf(-a2)
    return compound_p

S0 = 40
K = 40
T = 2
t = 1
r = 0.08
sigma = 0.30
delta = 0
x = 2
S_star = 35.89

Cp = compound_put(S0, K, T, t, r, sigma, delta, x, S_star)
print(f"After calculating in Python: Compound Put Price: ${Cp:.4f}")

After calculating in Python: Compound Put Price: $0.0783


## 21.13

Since the cash flows grow at the same rate as the risk-free rate during the initial 10 years, we can recognize that these cash flows are equivalent to the initial cash flow of $1. The cash flow after year 10 remains constant.
We want to find the present value formula for the project, we denote the time in which we invest $t^*$. The value of the cash flows at time $t^*$ is:

$$
PV_{t^*} = \sum_{i=1}^{10} \frac{1*(1.05)^{i-1}}{(1.05)^{i}} + \frac{1}{0.05}*\frac{(1.05)^{9}}{(1.05)^{10}} = 28.5714
$$

So at the moment of starting the project ($t^*$), the present value of benefits is \$28.5714. Therefore, the net present value at time zero is:

$$NPV(t^*) = \frac{PV_{t^*} - I}{(1+r)^{t^*}}$$

We should invest as soon as $PV \ge I$, therefore, the best is to invest immediately at $t=0$.

The value of investing at time $t=0$ is:
$NPV(0) = PV_0 - 20 =  8.5714$. The value of the option to invest is simply the NPV of the project, as it is the most optimal to start as soon as possible.



## 21.14

The land’s value is the option to extract the oil plus the residual value discounted back, like a perpetual American call option.

The option value is $V(S) = A S^\beta$ when $S < S^*$. First, we need $\beta$ from the quadratic equation $\frac{1}{2} \sigma^2 \beta (\beta - 1) + (r - \delta) \beta - r = 0$. Calculating with Python gives $\beta = 1.512430$ (the positive root).

Next, the optimal exercise threshold is $S^* = \frac{\beta (X - R)}{\beta - 1}$. With $X - R = 12.60$ and $\beta - 1 = 0.512430$, we get $S^* = 37.19$. So, extraction happens when the oil price hits $37.19$.

Then, we find $A$ using the value-matching condition $A (S^*)^\beta = S^ - X + R$. The payoff is $S^* - X + R = 24.59$, and $(S^*)^\beta = 237.21$, so $A = 0.103657$.

Finally, the land’s value at $S = 15$ is $V(S) = A S^\beta$. With $S^\beta = 60.08$, we compute $V(15)= 6.23$. Since $15 < 37.19$, we don’t extract now, and $6.23$ is the land’s value.

## Exam problem

Since the cash flows grow at the same rate as the risk-free rate during the initial 10 years, we can recognize that these cash flows are equivalent to the initial cash flow of $1. The cash flow after year 10 remains constant.
We want to find the present value formula for the project, we denote the time in which we invest $t^*$. The value of the cash flows at time $t^*$ is:

$$
PV_{t^*} = \sum_{i=1}^{10} \frac{1*(1.05)^{i-1}}{(1.05)^{i}} + \frac{1}{0.05}*\frac{(1.05)^{9}}{(1.05)^{10}} = 28.5714
$$

So at the moment of starting the project ($t^*$), the present value of benefits is \$28.5714. Therefore, the net present value at time zero is:

$$NPV(t^*) = \frac{PV_{t^*} - I}{(1+r)^{t^*}}$$

We should invest as soon as $PV \ge I$, therefore, the best is to invest immediately at $t=0$.

The value of investing at time $t=0$ is:
$NPV(0) = PV_0 - 20 =  8.5714$. The value of the option to invest is simply the NPV of the project, as it is the most optimal to start as soon as possible.



### a)

The price of a European gap call option can be computed using a variation of the Black-Scholes formula.

Given that there is no dividend, we set $q = 0$.  Substituting the provided values into the Black-Scholes formula:

$$d_1 = \frac{\ln(100/100) + (0.02 + 0.8^2/2) \cdot 2}{0.8 \sqrt{2}} \approx 0.6010$$

$$d_2 = d_1 - \sigma \sqrt{T} = 0.6010 - 0.8\sqrt{2} \approx -0.5303$$

$N(d_1) \approx 0.7261$ and $N(d_2) \approx 0.2979$.

Substituting these into the gap call option price formula:

$$ C = 100 \cdot 0.7261 - 150e^{-0.02\cdot2} \cdot 0.2979 \approx 29.6705$$

Therefore, the price of one gap call option is approximately 29.67.

### b)

The Black-Scholes formula for a plain vanilla call option is:

$$C_{vanilla} = S_0 e^{-qT} N(d_{1,vanilla}) - K e^{-rT} N(d_{2,vanilla})$$

where

$$d_{1,vanilla} = \frac{\ln(S_0/K) + (r - q + \sigma^2/2)T}{\sigma\sqrt{T}}$$

$$d_{2,vanilla} = d_{1,vanilla} - \sigma \sqrt{T}$$

The critical difference is that the gap call option uses the trigger price, $L$, in the calculation of $d_1$ and $d_2$, while the vanilla option uses the strike price, $K$. This leads to different values for $N(d_1)$ and $N(d_2)$, and therefore a different option price.

The gap call option price can be expressed as:

$$C_{gap} = S_0  N(d_1) - K e^{-rT}  N(d_2)$$

where $d_1$ and $d_2$ are calculated using $L$ (the trigger), as shown in part a. The vanilla call option, in contrast, would use $K$ in its $d_1$ and $d_2$ calculations. The core formula structure is the same; it is the inputs to the $d_1$ and $d_2$ terms that create the difference.